Firstly, the script reads data from the HR system. From the export, we select only those researchers which are currently employed by the university. 

In [1]:
import pandas as pd
import os
import re

bb_xlsx = os.path.join( 'November2022' , 'BB.xlsx' ) 
bb = pd.read_excel(bb_xlsx)
# Select active researchers; without duplicates
bb = bb[ bb['groep'] == 'in dienst' ]

bb = bb.drop_duplicates(['Medewerker - Email adres (Key)'])
bb['email'] = bb['Medewerker - Email adres (Key)'].str.lower()


print( f'The BB.xlsx file contains data about {bb.shape[0]} researchers.' )

The BB.xlsx file contains data about 3158 researchers.


Next, we read in the data exported from LUCRIS. 

In [2]:
#lucris_xlsx = os.path.join( 'Sept2022' , 'LUCRIS2.xlsx' ) 
lucris = pd.read_excel('LUCRIS_Oct2023.xlsx')
lucris = lucris.drop_duplicates(['email'])
lucris['email'] = lucris['email'].str.lower()

print( f'The export from LUCRIS contains data about {lucris.shape[0]} researchers.' )

The export from LUCRIS contains data about 30153 researchers.


In [3]:
lucris_emails = lucris['email'].to_list()

overlap = bb[ bb['email'].isin(lucris_emails) ]
missing = bb[ ~bb['email'].isin(lucris_emails) ]

print(f'{overlap.shape[0]} of the researchers in the LUCRIS export are also in the BB export.')
print(f'{missing.shape[0]} people in the BB export apparently stopped working for the UL.')

missing.to_csv('researchers_not_in_lucris.csv')

3007 of the researchers in the LUCRIS export are also in the BB export.
151 people in the BB export apparently stopped working for the UL.


The script creates dictionaries to connect the emails to the full names (i.e. combination of first and last names)

In [4]:
## find all emails, first names and last names

lucris_users = pd.read_csv('user_manager_export.csv')

first_name_dict = dict()
last_name_dict = dict()

for i,p in lucris_users.iterrows():
    email = str(p['Email']).strip()
    fn = str(p['First name']).strip()
    ln = str(p['Last name']).strip()
    first_name_dict[email] = fn
    last_name_dict[email] = ln


In [5]:
names_initials = dict()

for i,p in bb.iterrows():
    email = str(p['email']).strip()
    names_initials[email] = p['Medewerker'].strip()

Code to standardise the institute names. The institute names have not been used consistently in the BB export. 

In [6]:
# List of institute names
institutes = ['Biology', 'Chemistry', 'Computer Science', 'Mathematical Institute', 'LACDR', 'Physics', 'Astronomy', 'Public Administration', 'Global Affairs', 'Leiden University College',  'African Studies Centre Leiden', 'Milieuwetenschappen', 'Anthropology', 'Psychology', 'Pedagogische Wetenschappen', 'Politicologie', 'ICLON', 'CWTS',  'Performing Arts', 'LUCAS', 'History', 'LIAS', 'LUCL', 'Philosophy', 'Archaeology', 'Publiekrecht', 'Fiscale En Economische Vakken', 'Metajuridica', 'Privaatrecht', 'Strafrecht' ]

In [7]:

def replace_value(old,new):
    bb.loc[bb['Instituut'] == old, 'Instituut'] = new

replace_value('Geschiedenis' , 'History')
replace_value('Sterrewacht' , 'Astronomy')
replace_value('LIACS' , 'Computer Science')
replace_value('MI' , 'Mathematical Institute')
replace_value('Mathematisch Instituut' , 'Mathematical Institute')
replace_value('World Arch.' , 'Archaeology')
replace_value('Arch. Sciences' , 'Archaeology')
replace_value('Arch. Heritage' , 'Archaeology')
replace_value('Archeologie' , 'Archaeology')
replace_value('Wijsbegeerte' , 'Philosophy')
replace_value('ASC','African Studies Centre Leiden' )
replace_value('Psychologie ','Psychology')
replace_value('IIAS','LIAS')
replace_value('Intern. Studies','LIAS')
replace_value('NIMAR','LIAS')
replace_value('Area Studies ','LIAS')
replace_value('CML','Milieuwetenschappen')
replace_value('LIC','Chemistry')
replace_value('Chemisch Onderzoek','Chemistry')
replace_value('NeCEN','Biology')
replace_value('Biologie','Biology')
replace_value('Fiscaal-econom','Fiscale En Economische Vakken')
replace_value('Fiscale en Economische vakken','Fiscale En Economische Vakken')
replace_value('LION','Physics')
replace_value('Natuurkunde','Physics')
replace_value('Lucas','LUCAS')
replace_value('Cwts','CWTS')
replace_value('IBL','Biology')
replace_value('SGA','Global Affairs')
replace_value('Security and Global Affairs','Global Affairs')
replace_value('BSK','Public Administration')
replace_value('Bestuurskunde','Public Administration')
replace_value('LUC','Leiden University College')
replace_value('Kunsten','Performing Arts')
replace_value('CA/OS','Anthropology')
replace_value('Culturele Antropologie / OS','Anthropology')
replace_value('DPC','Dual Phd')
replace_value('Pedagogiek','Pedagogische Wetenschappen')
replace_value('VO','ICLON')
replace_value('HO','ICLON')
replace_value('Onderzoek','ICLON')
replace_value('FBS','Global Affairs')
replace_value('Linguistics','LUCL')
replace_value('Wetenschap en Technologie Studies','CWTS')
replace_value('Strafrecht en Criminologie','Strafrecht')
replace_value('Advanced Computer Science','Computer Science')
replace_value('Leiden University College The Hague','Leiden University College')
replace_value('UCL','Leiden University College')
replace_value('Politieke Wetenschap','Politicologie')
replace_value('Drug Research','LACDR')
replace_value('ASCL','African Studies Centre Leiden')
replace_value('Arts in Society','LUCAS')



In [8]:
## test if we are not missing any institutes

institutes_bb = list( set( bb['Instituut'].tolist() ) )

for i in institutes_bb:
    if i not in institutes:
        print(i)

Overig
Bestuursbureau
Honours Academy


Researchers are linked to ORCID ids using their email. This is taken from LUCRIS

In [9]:
## find names and emails in LUCRIS.xlxs
orcids = dict()

for i,p in lucris.iterrows():
    email = str(p['email']).strip().lower()
    orcid_id = str(p['ORCID']).strip().lower() 
    orcids[email] = orcid_id

We now have information on the full names and the ORCID ids from LUCRIS. This information is used to calculate percentages of researchers who have reguistered their ORCID is in LUCRIS. 

* 'orcid.csv' contains data about the percentages per institute. 
* 'no_orcis.csv' has information about the researchers without an ORCID ID in LUCRIS. The data set includes Institute, email and Full name. 
* 'names.csv' has the names and the emails of the registered researchers.

In [10]:
import re

def percentage(number):
    total_rounded = (number) * 100
    return round(total_rounded,2)

out = open('orcid.csv' , 'w')
out2 = open('no_orcid.csv' , 'w')
out3 = open( 'names.csv' , 'w' )

out.write('Institute,#Orcid,#Researchers,Percentage\n')
out2.write('institute,email,first_name,last_name\n')

total_ul_researchers = 0
total_ul_orcid = 0

for inst in institutes:
    out.write(f'\n{inst},')
    researchers = bb[ bb['Instituut'] == inst ]
    count_orcids = 0
    total = 0
    
    for i,row in researchers.iterrows():
        
        email = str(row['email']).strip()
        orcid = orcids.get(email,'')
        
        
        if email in last_name_dict and email in first_name_dict:
            first_name = first_name_dict[email]
            last_name = last_name_dict[email]
        else:
            full_name = email[ : email.index('@') ]
            if re.search( r'\.' , full_name):
                first_name = full_name[ : full_name.rindex('.')]
                last_name = full_name[ full_name.rindex('.')+1:]
            else:
                first_name = ''
                last_name = full_name
            
        
        try:

            if re.search( r'\d+' , orcid ):
                count_orcids += 1
                total_ul_orcid += 1
                out3.write(f"{inst},{email},{first_name},{last_name},{orcid}\n")
            else:
                out2.write(f"{inst},{email},{first_name},{last_name}\n")


            
            total += 1
            total_ul_researchers += 1
            
        except:
            print(f'Problem for {email}: Not in LUCRIS export')

            
        
    if total>0:
        out.write( f'{count_orcids},{total},{percentage(count_orcids/total)}' )

out.write( f'\nUL,{total_ul_orcid},{total_ul_researchers},{percentage(total_ul_orcid/total_ul_researchers)}' )            

out.close()
out2.close()
out3.close()


In [11]:
import urllib.request
import urllib.parse
import xml.etree.ElementTree as ET
import re
import pandas as pd
from orcidData import *
import os

import ssl                                        
openssl_dir, openssl_cafile = os.path.split(      
    ssl.get_default_verify_paths().openssl_cafile)


import ssl
ssl._create_default_https_context = ssl._create_unverified_context

ns = {'o': 'http://www.orcid.org/ns/orcid' ,
's' : 'http://www.orcid.org/ns/search' ,
'h': 'http://www.orcid.org/ns/history' ,
'p': 'http://www.orcid.org/ns/person' ,
'pd': 'http://www.orcid.org/ns/personal-details' ,
'a': 'http://www.orcid.org/ns/activities' ,
'e': 'http://www.orcid.org/ns/employment' ,
'c': 'http://www.orcid.org/ns/common' }


def getData( firstName , lastName , institute, email ):


    queryName = firstName + ' ' + lastName
    
    fullOutput = ''

    query = f'https://pub.orcid.org/v2.1/search?q=family-name:{ urlEncode(lastName) }+AND+given-names:{ urlEncode(firstName) }'
    print(query)

    root = getTree( query )
    hits = root.findall('s:result' , ns )


    if len(hits) == 0:
        queryName = urlEncode( queryName )
        query = "https://pub.orcid.org/v3.0/search?q=" + queryName
        root = getTree( query )
        hits = root.findall('s:result' , ns )

    count = 0
    for result in hits:
        try: 
            count += 1

            data = dict()
            orcidId = result.find('c:orcid-identifier/c:path' , ns ).text
            orcidUrl = "https://pub.orcid.org/v3.0/" + orcidId +  "/record"
            xml = getTree( orcidUrl )

            data['lastName'] = getLastName( xml )
            data['firstName'] = getFirstName( xml )
            data['creationDate'] = getCreationDate( xml )
            data['nrWorks'] = getNumberOfWorks( xml )
            aff = getAffiliations( xml )

            fullOutput += f"{ lastName }\t"
            fullOutput += f"{ firstName }\t"

            fullOutput += f"{ institute }\t"
            fullOutput += f"{ email}\t"

            institute, email
            fullOutput += f"{ orcidId }\t"
            fullOutput += f"{ data.get('lastName' , '' ) }\t"
            fullOutput += f"{ data.get('firstName' , '' ) }\t"
            fullOutput += f"{ data.get('creationDate' , '' ) }\t"
            fullOutput += f"{ data.get('nrWorks' , '' ) }\t"

            if len(aff) > 0:
                fullOutput += f"{ aff[0][0] }\t"
                fullOutput += f"{ aff[0][1] }\t"
            else:
                fullOutput += '\t\t'

            if len(aff) > 1:
                fullOutput += f"{ aff[1][0] }\t"
                fullOutput += f"{ aff[1][1] }\n"
            else:
                fullOutput += ',\n'
            if count == 3:
                break

        
        except:
            print(f'Problem with {lastName} {firstName}')
    return fullOutput


out = open( 'researchers_not_registered.tsv' , 'w' )
out.write( 'lastName\tfirstName\tinstitute\temail\torcid\tOrcidlastName\tOrcidfirstName\tcreationDate\tnrWorks\torganisation1\tdepartment1\torganisation2\tdepartment2\n' )



df = pd.read_csv( 'no_orcid.csv'  )
for index , column in df.iterrows():
    if index>=0:
        print(column['last_name'])
        if pd.notnull( column['last_name']) and pd.notnull( column['first_name']):

            if pd.isnull( column['first_name'] ):
                first_name = 'a'
            else:
                first_name = column['first_name']

            print( first_name , column['last_name'] , f'({column["institute"].strip()})')
            out.write( getData( urllib.parse.quote( first_name )  , urllib.parse.quote( column['last_name']  ) , column['institute'] , column['email']) )

out.close()


Slabbekoorn
Hans Slabbekoorn (Biology)
https://pub.orcid.org/v2.1/search?q=family-name:slabbekoorn+AND+given-names:hans


HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
nr = pd.read_csv('researchers_not_registered.tsv' , sep='\t')
nr.columns

In [ ]:
     
out = open('nr_leiden.csv','w',encoding='utf-8')
out2 = open('not_nr_leiden.csv','w',encoding='utf-8')

out.write('lastName\tfirstName\tinstitute\temail\torcid\tOrcidlastName\tOrcidfirstName\tcreationDate\tnrWorks\torganisation1\tdepartment1\torganisation2\tdepartment2\n')
out2.write('lastName\tfirstName\tinstitute\temail\torcid\tOrcidlastName\tOrcidfirstName\tcreationDate\tnrWorks\torganisation1\tdepartment1\torganisation2\tdepartment2\n')


for i,row in nr.iterrows():
    
    if re.search( r'Leiden',str(row['organisation1']),re.IGNORECASE) or re.search( r'Leiden',str(row['organisation2']),re.IGNORECASE) or re.search( r'Leiden',str(row['department1']),re.IGNORECASE) or re.search( r'Leiden',str(row['department2']),re.IGNORECASE):
        all_values = row.tolist()
        for value in all_values:
            out.write(f'{value}\t')
        out.write('\n')
    elif row['lastName'] == row['OrcidlastName'] and row['firstName'] == row['OrcidfirstName']:
        all_values = row.tolist()
        for value in all_values:
            out.write(f'{value}\t')
        out.write('\n')  
    else:
        all_values = row.tolist()
        for value in all_values:
            out2.write(f'{value}\t')
        out2.write('\n')

out.close()
out2.close()
        

Find the matches: Either an identical name, or an affiliation at Leiden. The result is saved as 'match.tsv'.

In [ ]:
df = pd.read_csv('nr_leiden.csv',sep='\t',index_col=False)
df['full_name'] = df['firstName'] + ' ' + df['lastName']
df = df.drop_duplicates(subset="full_name")
df.to_csv('nr_leiden_deduplicated.csv')

In [ ]:
df = pd.read_csv('nr_leiden_deduplicated.csv')
print(df.columns)
#print(df.iloc[0])


for inst in institutes:
    researchers = df[ df['institute'] == inst ]
    print(researchers.shape[0])


In [ ]:
matches = []

out1 = open( 'match.tsv' , 'w' , encoding = 'utf-8' )
out1.write('first_name\tlast_name\tinstitute\temail\torcid\torcid_first\torcid_last\tdate\tnr_works\torg1\tdept1\torg2\tdept2\n')
out2 = open( 'no_match.tsv' , 'w' , encoding = 'utf-8' )

df = pd.read_csv('researchers_not_registered.csv' , sep = '\t' )
cols = df.columns
for i,row in df.iterrows():
    full_name = row['lastName'] + ' ' + row['firstName']
    full_name = re.sub( r'\s+' , ' ' , full_name )
    orcid_name = f"{row['OrcidlastName']} {row['OrcidfirstName']}"
    orcid_name = re.sub( r'\s+' , ' ' , orcid_name )
    affiliation = f"{row['organisation1']} {row['department1']} {row['organisation2']} {row['department2']}"
    #print(affiliation)
    
    new_row = ''
    for c in cols: 
        new_row += f'{row[c]}\t'
    new_row = new_row.strip()
    
    if full_name == orcid_name:
        if full_name not in matches:
            matches.append(full_name)
            out1.write(f'{new_row}\n')
    elif re.search('leiden' , affiliation , re.IGNORECASE):
        if full_name not in matches:
            matches.append(full_name)
            out1.write(f'{new_row}\n')
    else:
        out2.write(f'{new_row}\n')
        
out1.close()
out2.close()

Totals included researchers not registered in LUCRIS

In [ ]:
df.head()

In [ ]:


out = open('name_not_registered.csv' , 'w' , encoding = 'utf-8')
out.write( 'first_name,last_name,institute,email\n')
        
## print names of all researchers who have not registered in 'researchers_not_registered.csv'     

df = pd.read_csv('match.tsv', sep = '\t')          
for i,row in df.iterrows():
    out.write(f"{row['first_name']},{row['last_name']},{row['institute']},{row['email']}\n")

out.close()

unregistered = dict()

for inst in institutes:
    researchers = df[ df['institute'] == inst ]
    print(researchers.shape[0])
    unregistered[inst] = researchers.shape[0]


out2 = open('orcid_including_non_registered.csv' , 'w' , encoding = 'utf-8')

total_ul_researchers = 0
total_ul_orcid = 0

for inst in institutes:
    out2.write(f'\n{inst},')
    researchers = bb[ bb['Instituut'] == inst ]
    total = len(researchers)
    total_ul_researchers += total
    count_orcids = 0
    for i,p in researchers.iterrows():
        
        email = str(p['Email']).strip().lower()
        orcid = orcids.get(email,'')
        try:
            if re.search( r'\d+' , orcid ):
                count_orcids += 1
                total_ul_orcid += 1
        except:
            print(f'Problem for {email}')

    count_orcids += unregistered[inst]
    total_ul_orcid += unregistered[inst]
    if total>0:
        out2.write( f'{count_orcids},{total},{count_orcids/total}' )
        

out2.write( f'\nUL,{total_ul_orcid},{total_ul_researchers},{total_ul_orcid/total_ul_researchers}' )            

out2.close()


In [ ]:
df1 = pd.read_csv('no_orcid.csv')
df2 = pd.read_csv('match.tsv' , sep = '\t')
has_orcid = list(df2['email'])

out = open('no_orcid_not_lucris.csv', 'w', encoding = 'utf-8')
for i,row in df1.iterrows():
    if row['email'] not in has_orcid:
        out.write(row['email'])
        out.write(',')
        out.write(row['first_name'])
        out.write(',')
        out.write(row['last_name'])
        out.write(',')
        out.write(row['institute'])
        out.write('\n')
out.close()